# Predicting points based on description (NLP) and other features with Catboost

In [1]:
pip install q nltk

In [2]:
!pip install catboost

     |████████████████████████████████| 64.8MB 59kB/s 


In [3]:
import nltk
nltk.download("popular")

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

True

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
cd

/root


In [6]:
%cd /content/drive/My Drive/Colab Notebooks/Wine recommender/Knight ML Assignment/Data

/content/drive/My Drive/Colab Notebooks/Wine recommender/Knight ML Assignment/Data


In [7]:
%ls

catboost_info/  test.csv  train.csv


In [0]:
import numpy as np
import pandas as pd

In [0]:
data=pd.read_csv('train.csv')

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82657 entries, 0 to 82656
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   user_name           63264 non-null  object 
 1   country             82622 non-null  object 
 2   review_title        82657 non-null  object 
 3   review_description  82657 non-null  object 
 4   designation         59010 non-null  object 
 5   points              82657 non-null  int64  
 6   price               77088 non-null  float64
 7   province            82622 non-null  object 
 8   region_1            69903 non-null  object 
 9   region_2            35949 non-null  object 
 10  winery              82657 non-null  object 
 11  variety             82657 non-null  object 
dtypes: float64(1), int64(1), object(10)
memory usage: 7.6+ MB


In [11]:
total = data.isnull().sum().sort_values(ascending = False)
percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
missing_data  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data

,Total,Percent
region_2,46708,56.508221
designation,23647,28.608587
user_name,19393,23.462018
region_1,12754,15.430030
price,5569,6.737481
province,35,0.042344
country,35,0.042344
variety,0,0.000000
winery,0,0.000000
points,0,0.000000


In [0]:
data=data.dropna(subset=['price'])

In [13]:
print("Total number of examples: ", data.shape[0])
print("Number of examples with the same title and description: ", data[data.duplicated(['review_description','review_title'])].shape[0])

Total number of examples:  77088
Number of examples with the same title and description:  4732


In [0]:
data=data.drop_duplicates(['review_description','review_title'])
data=data.reset_index(drop=True)

In [0]:
data=data.fillna(-1)

In [0]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
import string
from wordcloud import WordCloud, STOPWORDS
import re

from nltk.tokenize import RegexpTokenizer

In [17]:
data['review_description']= data['review_description'].str.lower()
data['review_description']= data['review_description'].apply(lambda elem: re.sub('[^a-zA-Z]',' ', elem))  
data['review_description']

0        classic chardonnay aromas of apple  pear and h...
1        this wine is near equal parts syrah and merlot...
2        barolo conca opens with inky dark concentratio...
3        it s impressive what a small addition of sauvi...
4        this ripe  sweet wine is rich and full of drie...
                               ...                        
72351    made in a drier  tarter style than the winery ...
72352    hot earth  baked cherry and mild vanilla aroma...
72353    pungent  high pitched notes of turmeric and dr...
72354    this is a wine that s not just lively and frui...
72355    a big  rich  vital cabernet with immediate att...
Name: review_description, Length: 72356, dtype: object

In [18]:
tokenizer = RegexpTokenizer(r'\w+')
words_descriptions = data['review_description'].apply(tokenizer.tokenize)
words_descriptions.head()

0    [classic, chardonnay, aromas, of, apple, pear,...
1    [this, wine, is, near, equal, parts, syrah, an...
2    [barolo, conca, opens, with, inky, dark, conce...
3    [it, s, impressive, what, a, small, addition, ...
4    [this, ripe, sweet, wine, is, rich, and, full,...
Name: review_description, dtype: object

In [19]:
all_words = [word for tokens in words_descriptions for word in tokens]
data['description_lengths']= [len(tokens) for tokens in words_descriptions]
VOCAB = sorted(list(set(all_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_words), len(VOCAB)))

3005587 words total, with a vocabulary size of 24598


In [20]:
from collections import Counter
count_all_words = Counter(all_words)
count_all_words.most_common(100)

[('and', 195443),
 ('the', 123934),
 ('a', 101546),
 ('of', 96372),
 ('with', 66979),
 ('this', 64031),
 ('is', 53536),
 ('it', 50525),
 ('wine', 44107),
 ('in', 36772),
 ('flavors', 36328),
 ('to', 32097),
 ('s', 30889),
 ('fruit', 27279),
 ('on', 25572),
 ('that', 22422),
 ('palate', 20593),
 ('aromas', 20064),
 ('from', 19210),
 ('finish', 19171),
 ('acidity', 18683),
 ('but', 17951),
 ('tannins', 17573),
 ('cherry', 17191),
 ('drink', 17178),
 ('black', 16575),
 ('ripe', 15084),
 ('are', 14521),
 ('has', 13565),
 ('for', 12674),
 ('red', 12336),
 ('by', 11320),
 ('oak', 10804),
 ('notes', 10477),
 ('dry', 10442),
 ('spice', 10267),
 ('as', 10180),
 ('its', 10078),
 ('now', 10035),
 ('an', 9973),
 ('rich', 9909),
 ('nose', 9350),
 ('berry', 9109),
 ('full', 9009),
 ('fresh', 8840),
 ('plum', 8147),
 ('blend', 8016),
 ('soft', 7672),
 ('sweet', 7569),
 ('well', 7553),
 ('fruits', 7413),
 ('cabernet', 7275),
 ('light', 7163),
 ('blackberry', 7152),
 ('apple', 7088),
 ('texture', 6926)

In [0]:
stopword_list = stopwords.words('english')
ps = PorterStemmer()
words_descriptions = words_descriptions.apply(lambda elem: [word for word in elem if not word in stopword_list])
words_descriptions = words_descriptions.apply(lambda elem: [ps.stem(word) for word in elem])
data['description_cleaned'] = words_descriptions.apply(lambda elem: ' '.join(elem))

In [22]:
all_words = [word for tokens in words_descriptions for word in tokens]
VOCAB = sorted(list(set(all_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_words), len(VOCAB)))
count_all_words = Counter(all_words)
count_all_words.most_common(100)

1828938 words total, with a vocabulary size of 17420


[('wine', 45620),
 ('flavor', 41092),
 ('fruit', 34920),
 ('finish', 22437),
 ('acid', 21500),
 ('aroma', 20885),
 ('palat', 20662),
 ('cherri', 20039),
 ('drink', 19350),
 ('tannin', 18888),
 ('black', 16597),
 ('ripe', 16119),
 ('dri', 15324),
 ('note', 13464),
 ('spice', 12866),
 ('red', 12466),
 ('rich', 12417),
 ('fresh', 11151),
 ('oak', 11132),
 ('show', 10514),
 ('berri', 10463),
 ('nose', 9359),
 ('blend', 9166),
 ('full', 9062),
 ('plum', 8916),
 ('textur', 8909),
 ('sweet', 8870),
 ('balanc', 8678),
 ('blackberri', 8643),
 ('bodi', 8490),
 ('offer', 8305),
 ('appl', 8083),
 ('soft', 7980),
 ('age', 7918),
 ('well', 7561),
 ('cabernet', 7393),
 ('light', 7377),
 ('structur', 7361),
 ('crisp', 7205),
 ('dark', 6798),
 ('raspberri', 6712),
 ('fruiti', 6595),
 ('herb', 6446),
 ('vanilla', 6342),
 ('give', 6298),
 ('hint', 6091),
 ('miner', 6030),
 ('pepper', 5933),
 ('year', 5931),
 ('citru', 5870),
 ('white', 5806),
 ('good', 5691),
 ('touch', 5679),
 ('juici', 5524),
 ('bright

In [0]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split
from catboost import Pool, CatBoostRegressor, cv

def prepare_dataframe(vect, data, features=True):
    vectorized=vect.fit_transform(data['description_cleaned']).toarray()
    vectorized=pd.DataFrame(vectorized)
    if features == True:
        X=data.drop(columns=['user_name','points','review_title','review_description','description_cleaned'])
        X=X.fillna(-1)
        print(X.columns)
        X=pd.concat([X.reset_index(drop=True),vectorized.reset_index(drop=True)],axis=1)
        categorical_features_indices =[0,1,3,4,5,6,7,8,9,10]
    else:
        X=vectorized
        categorical_features_indices =[]
    y=data['points']
    return X,y,categorical_features_indices

In [0]:
#model definintion and training.
def perform_model(X_train, y_train,X_valid, y_valid,X_test, y_test,categorical_features_indices,name):
    model = CatBoostRegressor(
        random_seed = 100,
        loss_function = 'RMSE',
        iterations=800,
    )
    
    model.fit(
        X_train, y_train,
        cat_features = categorical_features_indices,
        verbose=False,
        eval_set=(X_valid, y_valid)
    )
    
    print(name+" technique RMSE on training data: "+ model.score(X_train, y_train).astype(str))
    print(name+" technique RMSE on test data: "+ model.score(X_test, y_test).astype(str))
    

In [0]:
def prepare_variable(vect, data, features_append=True):
    X, y , categorical_features_indices = prepare_dataframe(vect, data,features_append)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, 
                                                        random_state=42)
    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, 
                                                        random_state=52)
    return X_train, y_train,X_valid, y_valid,X_test, y_test, categorical_features_indices

In [26]:
vect= CountVectorizer(analyzer='word', token_pattern=r'\w+',max_features=500)
training_variable=prepare_variable(vect, data)
perform_model(*training_variable, 'Bag of Words Counts')

Index(['country', 'designation', 'price', 'province', 'region_1', 'region_2',
       'winery', 'variety', 'description_lengths'],
      dtype='object')
Bag of Words Counts technique RMSE on training data: 0.8166571093350081
Bag of Words Counts technique RMSE on test data: 0.7820796951590161


In [27]:
vect= TfidfVectorizer(analyzer='word', token_pattern=r'\w+',max_features=500)
training_variable=prepare_variable(vect, data)


Index(['country', 'designation', 'price', 'province', 'region_1', 'region_2',
       'winery', 'variety', 'description_lengths'],
      dtype='object')


In [28]:
vect= CountVectorizer(analyzer='word', token_pattern=r'\w+',max_features=500)
training_variable=prepare_variable(vect, data, False)
perform_model(*training_variable, 'Bag of Words Counts')

Bag of Words Counts technique RMSE on training data: 0.6815098822033031
Bag of Words Counts technique RMSE on test data: 0.6245763466349764


In [29]:
vect= TfidfVectorizer(analyzer='word', token_pattern=r'\w+',max_features=500)
training_variable=prepare_variable(vect, data, False)
perform_model(*training_variable, 'TF-IDF')

TF-IDF technique RMSE on training data: 0.6913363079985584
TF-IDF technique RMSE on test data: 0.6268573789139431
